In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import AmazonGoogleBenchmark

benchmark = AmazonGoogleBenchmark(data_dir_path="../data/")
benchmark

14:28:47 INFO:Extracting Amazon-Google...
14:28:47 INFO:Reading Amazon-Google row_dict...
14:28:47 INFO:Reading Amazon-Google train.csv...
14:28:47 INFO:Reading Amazon-Google valid.csv...
14:28:47 INFO:Reading Amazon-Google test.csv...


<AmazonGoogleBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/amazon_google_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'manufacturer', 'price']

In [6]:
from tqdm.auto import tqdm
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_info_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'semantic_title': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'manufacturer': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'price': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    }
}

In [10]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=benchmark.row_dict)
row_numericalizer.attr_info_dict

14:28:47 INFO:For attr=title, computing actual max_str_len
14:28:47 INFO:For attr=title, using actual_max_str_len=26
14:28:47 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
14:28:50 INFO:For attr=manufacturer, computing actual max_str_len
14:28:50 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
14:28:50 INFO:For attr=manufacturer, using actual_max_str_len=16
14:28:50 INFO:For attr=price, computing actual max_str_len
14:28:50 INFO:actual_max_str_len=11 must be pair to enable NN pooling. Updating to 12
14:28:50 INFO:For attr=price, using actual_max_str_len=12


{'title': NumericalizeInfo(source_attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=26, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_title': NumericalizeInfo(source_attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 

In [11]:
datamodule = benchmark.build_pairwise_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=20,
    row_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

14:28:51 INFO:GPU available: True, used: True
14:28:51 INFO:TPU available: None, using: 0 TPU cores
14:28:51 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [14]:
trainer.fit(model, datamodule)

14:28:53 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 7.4 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
5.7 M     Trainable params
1.7 M     Non-trainable params
7.4 M     Total params


14:28:53 WARNING:true_pair_set has 82 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [15]:
model.blocker_net.get_signature_weights()

{'title': 0.40408727526664734,
 'semantic_title': 0.35267871618270874,
 'manufacturer': 0.12780879437923431,
 'price': 0.11543223261833191}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.420863309352518,
 'valid_f1_at_0.5': 0.7553366174055828,
 'valid_f1_at_0.7': 0.8340080971659919,
 'valid_f1_at_0.9': 0.7193877551020408,
 'valid_pair_entity_ratio_at_0.3': 1.9212253829321664,
 'valid_pair_entity_ratio_at_0.5': 0.8205689277899344,
 'valid_pair_entity_ratio_at_0.7': 0.5689277899343544,
 'valid_pair_entity_ratio_at_0.9': 0.34573304157549234,
 'valid_precision_at_0.3': 0.26651480637813213,
 'valid_precision_at_0.5': 0.6133333333333333,
 'valid_precision_at_0.7': 0.7923076923076923,
 'valid_precision_at_0.9': 0.8924050632911392,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 0.9829059829059829,
 'valid_recall_at_0.7': 0.8803418803418803,
 'valid_recall_at_0.9': 0.6025641025641025}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.3': 0.4152617568766637,
  'test_f1_at_0.5': 0.7240829346092504,
  'test_f1_at_0.7': 0.832,
  'test_f1_at_0.9': 0.7684729064039408,
  'test_pair_entity_ratio_at_0.3': 1.9370932754880694,
  'test_pair_entity_ratio_at_0.5': 0.8524945770065075,
  'test_pair_entity_ratio_at_0.7': 0.5770065075921909,
  'test_pair_entity_ratio_at_0.9': 0.37310195227765725,
  'test_precision_at_0.3': 0.2620380739081747,
  'test_precision_at_0.5': 0.5776081424936387,
  'test_precision_at_0.7': 0.7819548872180451,
  'test_precision_at_0.9': 0.9069767441860465,
  'test_recall_at_0.3': 1.0,
  'test_recall_at_0.5': 0.9700854700854701,
  'test_recall_at_0.7': 0.8888888888888888,
  'test_recall_at_0.9': 0.6666666666666666}]